In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: PI. Training: False. Training date: earlier
{'training': False, 'inferring': True, 'site': 'PI', 'training_date': 'earlier', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 15, 'get_multiple_metrics': False, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'log_filename': 'log_mcd_pi_earlier.pkl'}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 9; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 11; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 4
			elif config['training_date'] == 'earlier': exp = 5
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 13; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 6
			elif config['training_date'] == 'earlier': exp = 7
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 14; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 15; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

15

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


In [9]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [10]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [11]:
exp

15

In [12]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], repetition_n 10


In [13]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    for idx in range(repetition_n): # 10 repetitions
    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)


        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        # idx += 1
    

Beginning run number 0
manager.config {'training': False, 'inferring': True, 'site': 'PI', 'training_date': 'earlier', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 15, 'get_multiple_metrics': False, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'log_filename': 'log_mcd_pi_earlier.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/PI/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

[0.51986082 0.71240327 0.60813202 0.03000651 0.14305026 0.14121363
 0.06906595]
threshold:  0.4693407249983777
threshold 0.4693407249983777


ic| label_current_deforestation_test_classified_incorrect.shape: (3676371,)
    predicted_test_classified_incorrect.shape: (3676371,)
ic| TP_H + FN_H + FP_H + TN_H: 3676371
    len(label_mask_current_deforestation_test): 122544147
ic| TP_H + FN_H + FP_H + TN_H: 3676371
    len(label_mask_current_deforestation_test): 122544147


[0.519821   0.71240449 0.60815643 0.03000038 0.14301646 0.14107845
 0.06904123]
threshold 0.4693407249983777
(122544147,) (122544147,)


ic| self.m_optimal: {'AA': array([0.03000038]),
                     'UEO': array([0.06904123]),
                     'f1': 54.34,
                     'f1_H': array([0.14204085]),
                     'f1_L': array([0.60106339]),
                     'precision_H': array([0.14301646]),
                     'precision_L': array([0.519821]),
                     'recall_H': array([0.14107845]),
                     'recall_L': array([0.71240449]),
                     'recall_Ltotal': array([0.60815643])}
ic| self.m_audited_optimal: {'f1': array([0.65146989]),
                             'precision': array([0.57320369]),
                             'recall': array([0.75448911])}


cm_audited [[120497170    736306]
 [   321784    988887]]
[0.57320369 0.75448911]
Result idx 3: {'uncertainty_result': {'metrics': {'precision_L': array([0.519821]), 'recall_L': array([0.71240449]), 'recall_Ltotal': array([0.60815643]), 'AA': array([0.03000038]), 'precision_H': array([0.14301646]), 'recall_H': array([0.14107845]), 'UEO': array([0.06904123]), 'f1_L': array([0.60106339]), 'f1_H': array([0.14204085]), 'f1': 54.34}, 'metrics_audited': {'precision': array([0.57320369]), 'recall': array([0.75448911]), 'f1': array([0.65146989])}, 'exp': 15}}
Grid execution idx: 3


In [14]:
error_count

0

In [15]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.53195804]), 'recall_L': array([0.69958021]), 'recall_Ltotal': array([0.61956172]), 'AA': array([0.0299989]), 'precision_H': array([0.10644846]), 'recall_H': array([0.26529932]), 'UEO': array([0.09367737]), 'f1_L': array([0.60436177]), 'f1_H': array([0.15193476]), 'f1': 53.06}, 'metrics_audited': {'precision': array([0.57381291]), 'recall': array([0.73394245]), 'f1': array([0.64407406])}, 'exp': 15}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.44728461]), 'recall_L': array([0.65520344]), 'recall_Ltotal': array([0.57612883]), 'AA': array([0.03000089]), 'precision_H': array([0.10036852]), 'recall_H': array([0.35776325]), 'UEO': array([0.12158722]), 'f1_L': array([0.53163826]), 'f1_H': array([0.15675912]), 'f1': 45.57}, 'metrics_audited': {'precision': array([0.49463568]), 'recall': array([0.69681594]), 'f1': array([0.57857158])}, 'exp': 15}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.64663261])

In [16]:
print(len(results))

4


In [17]:
results[0]

{'uncertainty_result': {'metrics': {'precision_L': array([0.53195804]),
   'recall_L': array([0.69958021]),
   'recall_Ltotal': array([0.61956172]),
   'AA': array([0.0299989]),
   'precision_H': array([0.10644846]),
   'recall_H': array([0.26529932]),
   'UEO': array([0.09367737]),
   'f1_L': array([0.60436177]),
   'f1_H': array([0.15193476]),
   'f1': 53.06},
  'metrics_audited': {'precision': array([0.57381291]),
   'recall': array([0.73394245]),
   'f1': array([0.64407406])},
  'exp': 15}}

In [18]:
manager.f1

54.34

In [19]:
print(manager.f1)

54.34
